In [1]:
import sys
sys.path.append("C:/Users/parspooya/anaconda3/envs/myNewEnv/Lib/site-packages")

import numpy as np
import pandas as pd
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import SAC
import gym
from gym import spaces
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure

In [2]:
# بارگذاری دیتاست
data = pd.read_excel('Book5.xlsx')

In [10]:
# داده‌های ورودی و خروجی
X = data[['p1','k1','s1','t1','a1','p2','k2','s2','t2','a2','p3','k3','s3','t3','a3','p4','k4','s4','t4','a4','p5','k5','s5','t5','a5']].values
y = data['task_variant'].values

In [11]:
# تقسیم داده‌ها به داده‌های آموزشی و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# تعریف محیط 
class CustomEnv(gym.Env):
    def __init__(self, X, y):
        super(CustomEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_step = 0
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)  # فضای عمل پیوسته
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(25,), dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        return self.X[self.current_step]
    
    def step(self, action):
        action = np.clip(action, 0, 2)  # مقادیر عمل را به محدوده 0 تا 2 تغییر می‌دهیم
        reward = 1 if int(action) + 1 == self.y[self.current_step] else 0
        self.current_step += 1
        done = self.current_step >= len(self.X)
        obs = self.X[self.current_step] if not done else np.zeros(self.observation_space.shape)
        return obs, reward, done, {}
    
    def render(self, mode='human'):
        pass

In [9]:
del model

In [13]:
# ساخت وکتور محیط و آموزش مدل SAC
env = DummyVecEnv([lambda: CustomEnv(X_train, y_train)])

model = SAC(
    'MlpPolicy', 
    env, 
    tensorboard_log='./dqn_tensorboard/' ,
    verbose=1
)


model.learn(total_timesteps=100000, log_interval=4)
# model.save("sac_25enters")


Using cpu device
Logging to ./dqn_tensorboard/SAC_3


C:\Users\parspooya\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 27       |
|    time_elapsed    | 1152     |
|    total_timesteps | 32000    |
| train/             |          |
|    actor_loss      | -40.8    |
|    critic_loss     | 0.229    |
|    ent_coef        | 0.0199   |
|    ent_coef_loss   | 0.111    |
|    learning_rate   | 0.0003   |
|    n_updates       | 31899    |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 28       |
|    time_elapsed    | 2251     |
|    total_timesteps | 64000    |
| train/             |          |
|    actor_loss      | -47.3    |
|    critic_loss     | 0.287    |
|    ent_coef        | 0.0184   |
|    ent_coef_loss   | -0.665   |
|    learning_rate   | 0.0003   |
|    n_updates       | 63899    |
---------------------------------
---------------------------------
| time/       

In [14]:
# ارزیابی مدل
env = CustomEnv(X_test, y_test)
obs = env.reset()
correct_predictions = 0
total_predictions = 0

# لیست پیش‌بینی‌ها و برچسب‌های واقعی
all_predictions = []
all_labels = []

while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    correct_predictions += reward
    total_predictions += 1
    all_predictions.append(int(action) + 1)
    all_labels.append(env.y[env.current_step - 1])
    
    if done:
        obs = env.reset()
        break

# ارزیابی
accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'F1 Score: {f1:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Accuracy: 50.35%
F1 Score: 0.34
Precision: 0.25
Recall: 0.50


C:\Users\parspooya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
